In [1]:
import bs4
import pandas as pd
import requests
import sqlite3
import time

In [2]:
def passed_time(start):
    sec = time.time() - start
    hr = sec // (60*60)
    sec = sec - (hr*60*60)
    mn = sec // (60)
    sec = sec - (mn*60)
    return f'{int(hr)}:{int(mn)}:{int(sec)}'

In [3]:
try:
    con = sqlite3.connect('/Users/qlanners/projects/mlb_salary_comp/sqlite_db/mlb_salary_comp.db')
except:
    print('Error connecting to db.')
    exit()

In [4]:
years = [2019]
url_gamelog = 'https://www.baseball-reference.com/players/gl.fcgi?id=player_id&t=gamelog_type&year=year_num'
gamelog_types = {'b':'batting', 'f':'fielding', 'p':'pitching'}

start_time = time.time()
for year in years:
    print(f'Starting {year}{"."*100}')
    url_L = f"https://www.baseball-reference.com/leagues/MLB/{year}.shtml"
    res_L = requests.get(url_L)
    soup_L=bs4.BeautifulSoup(res_L.text, 'html.parser')
    body_L = soup_L.find('tbody')
    teams_urls = [link['href'] for link in body_L.findAll('a', href=True)]
    
    for url_team in teams_urls:
        url_T = 'https://www.baseball-reference.com' + url_team
        res_T = requests.get(url_T)
        soup_T=bs4.BeautifulSoup(res_T.text, 'html.parser')
        batters = [link['href'] for link in soup_T.find('table', {'id': 'team_batting'}).find('tbody').findAll('a', href=True)]
        pitchers = [link['href'] for link in soup_T.find('table', {'id': 'team_pitching'}).find('tbody').findAll('a', href=True)]
        players = set([p.split('.shtml')[0].split('/')[-1] for p in batters+pitchers])
        for p in players:
            print(f'{p}: {year}')
            for k,v in gamelog_types.items():
                attempts = 0
                try:
                    while attempts < 3:
                        try:
                            games_table = pd.read_html(requests.get(url_gamelog.replace('player_id', p).replace('gamelog_type', k).replace('year_num', str(year)), timeout=10).text, header=0)
                            attempts = 99
                        except requests.exceptions.RequestException:
                            attempts += 1
                            if attempts == 3:                        
                                raise ValueError
                except ValueError:
                    print(f'-{k} (Loading URL Table)')
                    continue                    
                try:
                    if k == 'f':
                        games_table = games_table[0]
                    else:
                        games_table = games_table[-1]
                except:
                    print(f'-{k} (Table Index)')
                    continue
                if k == 'b':
                    if 'OBP' not in games_table.columns:
                        print(f'-{k} (Wrong Table)')
                        continue
                elif k == 'f':
                    if 'E' not in games_table.columns:
                        print(f'-{k} (Wrong Table)')
                        continue
                elif k == 'p':
                    if 'IP' not in games_table.columns:
                        print(f'-{k} (Wrong Table)')
                        continue
                try:
                    games_table = games_table[games_table['Tm'] != 'Tm']
                    games_table.dropna(subset=['Rk'], inplace=True)
                    games_table.reset_index(inplace=True)
                    games_table.insert(0, 'year', year)                
                    games_table.insert(0, 'player_id', p)
                except KeyError:
                    print(f'-{k} (Editing Table)')
                    continue
                try:
                    games_table.to_sql(f'games_{v}', con, if_exists='append', index=False)
                    print(f'.{k}')
                except:
                    print(f'-{k} (DB)')
        print(f'Time Elapsed: {passed_time(start_time)}')
                    
    print(f'Finished {year}')
    print(f'Time Elapsed: {passed_time(start_time)}')
    print("."*100)
    print()


Starting 2019....................................................................................................
josepca01: 2019


/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


.b
.f
.p
greinza01: 2019
.b
.f
.p
vargail01: 2019
.b
.f
-p (Loading URL Table)
leybado01: 2019
.b
.f
-p (Loading URL Table)
rayro02: 2019
.b
.f
.p
scottro02: 2019
.b
.f
.p
peralda01: 2019
.b
.f
-p (Loading URL Table)
ginkeke01: 2019
.b
.f
.p
tomasya01: 2019
.b
-f (Loading URL Table)
-p (Loading URL Table)
rojasjo01: 2019
.b
.f
-p (Loading URL Table)
kochma01: 2019
.b
.f
.p
payamjo01: 2019
.b
.f
.p
kellyme01: 2019
.b
.f
.p
hollagr01: 2019
.b
.f
.p
florewi01: 2019
.b
.f
-p (Loading URL Table)
andrima01: 2019
.b
.f
.p
sherfji01: 2019
.b
.f
.p
ahmedni01: 2019
.b
.f
-p (Loading URL Table)
murphjr01: 2019
.b
.f
.p
marteke01: 2019
.b
.f
-p (Loading URL Table)
crichst01: 2019
.b
.f
.p
almonab01: 2019
.b
.f
-p (Loading URL Table)
walkech02: 2019
.b
.f
-p (Loading URL Table)
bradlar01: 2019
.b
.f
.p
walketa01: 2019
.b
.f
.p
weavelu01: 2019
.b
.f
.p
mcfartj01: 2019
.b
.f
.p
lopezyo01: 2019
.b
.f
.p
youngal01: 2019
.b
.f
.p
clarkta01: 2019
.b
.f
.p
dysonja01: 2019
.b
.f
-p (Loading URL Table)
avil

.b
.f
.p
zagunma01: 2019
.b
.f
-p (Loading URL Table)
rizzoan01: 2019
.b
.f
-p (Loading URL Table)
davista01: 2019
.b
.f
.p
stroppe01: 2019
.b
.f
.p
garciro02: 2019
.b
.f
-p (Loading URL Table)
millsal02: 2019
.b
.f
.p
mapledi01: 2019
.b
.f
.p
phelpda01: 2019
.b
.f
.p
hamelco01: 2019
.b
.f
.p
contrwi01: 2019
.b
.f
-p (Loading URL Table)
maldoma01: 2019
.b
.f
-p (Loading URL Table)
happia01: 2019
.b
.f
-p (Loading URL Table)
Time Elapsed: 0:6:54
delmoni01: 2019
.b
.f
-p (Loading URL Table)
palkada01: 2019
.b
.f
-p (Loading URL Table)
fulmeca01: 2019
.b
.f
.p
osichjo01: 2019
.b
.f
.p
bummeaa01: 2019
.b
.f
.p
mendida01: 2019
.b
.f
-p (Loading URL Table)
garcile02: 2019
.b
.f
-p (Loading URL Table)
banuema01: 2019
.b
.f
.p
detwiro01: 2019
.b
.f
.p
engelad01: 2019
.b
.f
-p (Loading URL Table)
rodonca01: 2019
-b (Loading URL Table)
.f
.p
frareca01: 2019
-b (Loading URL Table)
.f
.p
jonesna01: 2019
-b (Loading URL Table)
.f
.p
fryja01: 2019
.b
.f
.p
santihe01: 2019
.b
.f
.p
ceasedy01: 2019
-b

.b
.f
.p
adamsau01: 2019
.b
.f
.p
goodrni01: 2019
.b
.f
-p (Loading URL Table)
hicksjo02: 2019
.b
.f
-p (Loading URL Table)
garrere01: 2019
-b (Loading URL Table)
.f
.p
jimened01: 2019
-b (Loading URL Table)
.f
.p
sotogr01: 2019
.b
.f
.p
peterdu01: 2019
.b
.f
-p (Loading URL Table)
jonesja07: 2019
.b
.f
-p (Loading URL Table)
candeje01: 2019
.b
.f
-p (Loading URL Table)
castrwi01: 2019
.b
.f
-p (Loading URL Table)
wilsobo02: 2019
.b
.f
-p (Loading URL Table)
cabremi01: 2019
.b
.f
-p (Loading URL Table)
baezsa01: 2019
-b (Loading URL Table)
.f
.p
mercejo03: 2019
.b
.f
-p (Loading URL Table)
garcibr01: 2019
-b (Loading URL Table)
.f
.p
greensh02: 2019
.b
.f
.p
lugoda01: 2019
.b
.f
-p (Loading URL Table)
jacksed01: 2019
.b
.f
.p
rodriro03: 2019
.b
.f
-p (Loading URL Table)
mckayda02: 2019
-b (Loading URL Table)
.f
.p
dixonbr01: 2019
.b
.f
.p
demertr01: 2019
.b
.f
-p (Loading URL Table)
harrijo05: 2019
.b
.f
-p (Loading URL Table)
fernajo04: 2019
-b (Loading URL Table)
.f
.p
hardybl01: 201

.b
.f
-p (Loading URL Table)
kellyjo05: 2019
.b
.f
.p
sadleca02: 2019
.b
.f
.p
maedake01: 2019
.b
.f
.p
barneau01: 2019
.b
.f
-p (Loading URL Table)
Time Elapsed: 0:20:4
pradoma01: 2019
.b
.f
-p (Loading URL Table)
riverya01: 2019
.b
.f
-p (Loading URL Table)
steckdr01: 2019
.b
.f
.p
noesihe01: 2019
.b
.f
.p
yamamjo01: 2019
.b
.f
.p
holadbr01: 2019
.b
.f
.p
brighje01: 2019
.b
.f
.p
smithca03: 2019
.b
.f
.p
castiwi01: 2019
.b
.f
-p (Loading URL Table)
briceau01: 2019
.b
.f
.p
conlead01: 2019
.b
.f
.p
castrst01: 2019
.b
.f
-p (Loading URL Table)
hernael01: 2019
.b
.f
.p
duggero01: 2019
.b
.f
.p
romose01: 2019
.b
.f
.p
diazis01: 2019
.b
.f
-p (Loading URL Table)
kinlety01: 2019
.b
.f
.p
puellce01: 2019
.b
.f
-p (Loading URL Table)
kelleky01: 2019
.b
.f
.p
obriepe01: 2019
.b
.f
-p (Loading URL Table)
walkene01: 2019
.b
.f
-p (Loading URL Table)
herrero02: 2019
.b
.f
-p (Loading URL Table)
wallach01: 2019
.b
.f
-p (Loading URL Table)
bertijo01: 2019
.b
.f
-p (Loading URL Table)
smithjo08: 2

-b (Loading URL Table)
.f
.p
higasky01: 2019
.b
.f
-p (Loading URL Table)
gardnbr01: 2019
.b
.f
-p (Loading URL Table)
paxtoja01: 2019
.b
.f
.p
judgeaa01: 2019
.b
.f
-p (Loading URL Table)
adamsch01: 2019
-b (Loading URL Table)
.f
.p
happja01: 2019
.b
.f
.p
dullry01: 2019
-b (Loading URL Table)
.f
.p
rosaad01: 2019
-b (Loading URL Table)
.f
.p
cessalu01: 2019
.b
.f
.p
montgjo01: 2019
-b (Loading URL Table)
.f
.p
lyonsty01: 2019
.b
.f
.p
betande01: 2019
-b (Loading URL Table)
.f
.p
germado01: 2019
.b
.f
.p
encared01: 2019
.b
.f
-p (Loading URL Table)
gearrco01: 2019
.b
.f
.p
estrath01: 2019
.b
.f
-p (Loading URL Table)
tarplst01: 2019
-b (Loading URL Table)
.f
.p
greench03: 2019
.b
.f
.p
severlu01: 2019
-b (Loading URL Table)
.f
.p
ottavad01: 2019
.b
.f
.p
kahnlto01: 2019
.b
.f
.p
lemahdj01: 2019
.b
.f
-p (Loading URL Table)
sanchga02: 2019
.b
.f
-p (Loading URL Table)
tanakma01: 2019
.b
.f
.p
chapmar01: 2019
.b
.f
.p
gregodi01: 2019
.b
.f
-p (Loading URL Table)
tauchmi01: 2019
.b
.f
-p

.b
.f
.p
dickeal01: 2019
.b
.f
-p (Loading URL Table)
tatisfe02: 2019
.b
.f
-p (Loading URL Table)
strahma01: 2019
.b
.f
.p
Time Elapsed: 0:32:45
bruceja01: 2019
.b
.f
-p (Loading URL Table)
courtry01: 2019
.b
.f
-p (Loading URL Table)
fraleja01: 2019
.b
.f
-p (Loading URL Table)
swarzan01: 2019
.b
.f
.p
santado01: 2019
.b
.f
-p (Loading URL Table)
crawfjp01: 2019
.b
.f
-p (Loading URL Table)
broxtke01: 2019
.b
.f
-p (Loading URL Table)
freitda01: 2019
.b
.f
-p (Loading URL Table)
warrear01: 2019
-b (Loading URL Table)
.f
.p
eliasro01: 2019
.b
.f
.p
murphto04: 2019
.b
.f
.p
narvaom01: 2019
.b
.f
-p (Loading URL Table)
biddlje01: 2019
.b
.f
.p
bautige01: 2019
-b (Loading URL Table)
.f
.p
bassan01: 2019
.b
.f
.p
sheffju01: 2019
.b
.f
.p
mooredy01: 2019
.b
.f
.p
festama01: 2019
.b
.f
.p
rosscza01: 2019
.b
.f
.p
healyry01: 2019
.b
.f
-p (Loading URL Table)
longsh01: 2019
.b
.f
-p (Loading URL Table)
smithma05: 2019
.b
.f
-p (Loading URL Table)
hernafe02: 2019
.b
.f
.p
milonto01: 2019
.b
.f

.f
.p
Time Elapsed: 0:38:55
allarko01: 2019
.b
.f
.p
burkebr01: 2019
-b (Loading URL Table)
.f
.p
kineris01: 2019
.b
.f
-p (Loading URL Table)
heinesc01: 2019
.b
.f
-p (Loading URL Table)
smylydr01: 2019
.b
.f
.p
farrelu01: 2019
-b (Loading URL Table)
.f
.p
lynnla01: 2019
.b
.f
.p
biddlje01: 2019
.b
.f
.p
montera01: 2019
.b
.f
.p
chaveje01: 2019
.b
.f
.p
gallojo01: 2019
.b
.f
-p (Loading URL Table)
federti01: 2019
.b
.f
.p
forsylo01: 2019
.b
.f
-p (Loading URL Table)
huangwe01: 2019
-b (Loading URL Table)
.f
.p
hearnta01: 2019
-b (Loading URL Table)
.f
.p
martich02: 2019
.b
.f
.p
volqued01: 2019
-b (Loading URL Table)
.f
.p
cabreas01: 2019
.b
.f
-p (Loading URL Table)
palumjo01: 2019
-b (Loading URL Table)
.f
.p
millesh01: 2019
.b
.f
.p
santada01: 2019
.b
.f
-p (Loading URL Table)
mazarno01: 2019
.b
.f
-p (Loading URL Table)
trevijo01: 2019
.b
.f
-p (Loading URL Table)
valdeph01: 2019
-b (Loading URL Table)
.f
.p
mendeyo01: 2019
-b (Loading URL Table)
.f
.p
gibauia01: 2019
-b (Loading 